<img src="https://github.com/PKhosravi-CityTech/LightCnnRad/raw/main/Images/BioMindLogo.png" alt="BioMind AI Lab Logo" width="150" height="150" align="left" style="margin-bottom: 40px;"> **Repository Developed by Pegah Khosravi, Principal Investigator of the BioMind AI Lab**

Welcome to this repository! This repository is a result of collaborative efforts from our dedicated team at the lab. We are committed to advancing the field of biomedical AI and pushing the boundaries of medical data analysis. Your interest and contributions to our work are greatly appreciated. For more information about our lab and ongoing projects, please visit the [BioMind AI Lab website](https://sites.google.com/view/biomind-ai-lab). Thank you for your interest and support!


Importing the necessary libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
import glob
import numpy as np
from torch.autograd import Variable
import csv
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix
import seaborn as sns
import random

Checking the availibility of GPU for analysis

In [ ]:
torch.cuda.empty_cache()

# Set device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

In [ ]:
# Set seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Optional: Ensures that every run on the same machine yields the same result
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = False

Defining transforms and dataloaders and training the VGG-16 model from scratch.

In [ ]:
# Set data directory
data_dir = "Datasets/Alzheimer MRI"


# Define data transformations
data_transforms = {
    'Train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5],  # Normalizes pixel values to [-1, 1]
                             [0.5, 0.5, 0.5])
    ]),
    'Validation': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5],  # Normalizes pixel values to [-1, 1]
                             [0.5, 0.5, 0.5])
    ]),
    'Test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5],  # Normalizes pixel values to [-1, 1]
                             [0.5, 0.5, 0.5])
    ]),
}



# Defining classes
classes = os.listdir(os.path.join(data_dir, 'Test'))


# Create data loaders
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['Train', 'Validation']
}


dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
    for x in ['Train', 'Validation']
}


# Load the VGG16 model without pre-trained weights
model = models.vgg16(pretrained=False)

# Modify the classifier for binary classification (2 classes)
model.classifier[6] = nn.Linear(4096, 2)

# Send the model to the selected device (CPU or GPU)
model = model.to(device)

# Initialize the weights of the model (optional but recommended when training from scratch)
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

model.apply(weights_init)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

accuracies_dic = {'Train': [], 'Validation': []}
losses_dic = {'Train': [], 'Validation': []}

# Training loop
def train_model(model, dataloaders, criterion, optimizer, num_epochs=100):
    # Model training and saving best model
    best_accuracy = 0.0
    patience = 5  # Number of epochs to wait after last time validation accuracy improved.
    patience_counter = 0  # Counter to track the number of epochs since the last improvement
    for epoch in range(num_epochs):
        for phase in ['Train', 'Validation']:
            if phase == 'Train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'Train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'Train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item()
                corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / len(dataloaders[phase])
            epoch_acc = corrects.double() / len(dataloaders[phase].dataset)
            losses_dic[phase].append(epoch_loss)
            accuracies_dic[phase].append(epoch_acc.cpu().numpy().item())

            print(f'Epoch {epoch + 1}/{num_epochs} - {phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            if phase == 'Validation':
                if epoch_acc > best_accuracy:
                    torch.save(model.state_dict(), 'best_checkpoint.model')
                    best_accuracy = epoch_acc
                    patience_counter = 0  # Reset counter
                else:
                    patience_counter += 1

        if patience_counter > patience:
            print("Early stopping triggered.")
            break  # Exit from the training loop

# Train the model
train_model(model, dataloaders, criterion, optimizer, num_epochs=100)

Loading the trained-from-scratch VGG-16 model

In [ ]:
checkpoint=torch.load("best_checkpoint.model")
model.load_state_dict(checkpoint)
model.eval()

Evaluation of test set for the trained-from-scratch VGG-16 (Image-based).

In [ ]:
def prediction(img_path, transformer, model):
    model.eval()
    image = Image.open(img_path)
    image_tensor = transformer(image).float()
    image_tensor = image_tensor.unsqueeze_(0)

    if torch.cuda.is_available():
        image_tensor = image_tensor.cuda()

    with torch.no_grad():  # No need to compute gradients during inference
        input = Variable(image_tensor)
        output = model(input)

    prob = nn.functional.softmax(output, dim=1)
    prob = prob.cpu().detach().numpy()  # Move the tensor to CPU and then convert to numpy
    return prob

classes = os.listdir(os.path.join(data_dir, 'Test'))
if torch.cuda.is_available():
    model.cuda()

pred_path = os.path.join(data_dir, 'Test')

images_path = glob.glob(pred_path + '/**/*.png')
pred_dict = {}

for i in images_path:
    filename = i[i.rfind('/') + 1:]
    prob_array = prediction(i, data_transforms['Test'], model)
    pred_dict[filename] = prob_array

# Print the probabilities in the desired format
for filename, prob_array in pred_dict.items():
    print(f"'{filename}': np.array({np.array2string(prob_array, separator=', ', formatter={'float_kind': lambda x: f'{x:.8e}'} )}, dtype=np.float32),")


# Define the path for saving the CSV file
output_csv_path = 'train_from_scratch_predictions.csv'

# Save prediction results to CSV
with open(output_csv_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Label', 'Predicted Class', 'Probability'])

    for filename, prob_array in pred_dict.items():
        predicted_class = classes[1] if prob_array[0][1] > 0.5 else classes[0]  # Adjust the threshold as needed
        probability = prob_array[0][1]
        csvwriter.writerow([filename.split('\\')[-2], predicted_class, probability])

print("Prediction results saved to", output_csv_path)


Evaluation of test set for the trained-from-scratch VGG-16 (Patient-based)

In [ ]:
# Prediction function
def prediction(img_path, transformer, model):
    model.eval()
    image = Image.open(img_path)
    image_tensor = transformer(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor = image_tensor.cuda()

    with torch.no_grad():  # No need to compute gradients during inference
        input = Variable(image_tensor)
        output = model(input)

    prob = nn.functional.softmax(output, dim=1)
    prob = prob.cpu().detach().numpy()  # Move the tensor to CPU and then convert to numpy
    return prob


if torch.cuda.is_available():
    model.cuda()

pred_path = os.path.join(data_dir, 'Test')

images_path = glob.glob(pred_path + '/**/*.png')
pred_dict = {}

for i in images_path:
    filename = i[i.rfind('/') + 1:]
    prob_array = prediction(i, data_transforms['Test'], model)
    pred_dict[filename] = prob_array

# Print the probabilities in the desired format
for filename, prob_array in pred_dict.items():
    print(f"'{filename}': np.array({np.array2string(prob_array, separator=', ', formatter={'float_kind': lambda x: f'{x:.8e}'} )}, dtype=np.float32),")


# Set the desired threshold for considering predictions as positive
positive_threshold = 0.5

# Define the path for saving the CSV file
output_csv_path = 'train_from_scratch_predictions_vote.csv'

# Dictionary to store patient-wise predictions
patient_predictions = {}

# Populate the dictionary with individual image predictions
for filename, prob_array in pred_dict.items():
    patient_id = "_".join(filename.split('_')[:-1]) 
    prediction = prob_array[0][1]

    if patient_id not in patient_predictions:
        patient_predictions[patient_id] = []

    patient_predictions[patient_id].append(prediction)

# Calculate majority vote and save results to CSV
with open(output_csv_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Label', 'Predicted Class', 'Probability'])

    for patient_id, predictions in patient_predictions.items():
        majority_vote = sum(predictions) / len(predictions)
        predicted_class = classes[1] if majority_vote > positive_threshold else classes[0]
        if majority_vote >= positive_threshold or (1 - majority_vote) >= positive_threshold:
            csvwriter.writerow([patient_id.split('\\')[-2], predicted_class, majority_vote])  # Writing patient ID and majority vote prediction

print("Prediction results saved to", output_csv_path)


Plotting ROC curve for the VGG-16 model trained from scratch.

In [ ]:
# Load data from predictions.csv
df = pd.read_csv('train_from_scratch_predictions_vote.csv')

# Extract true labels and predicted probabilities
true_labels = df['Label']
predicted_probs = df['Probability']

# Convert labels to binary format (0 for 'N_' and 1 for 'Y_')
true_labels = true_labels.apply(lambda x: 0 if x.startswith(classes[0]) else 1)

# Calculate ROC curve and AUC
fpr, tpr, thresholds = roc_curve(true_labels, predicted_probs)
roc_auc = auc(fpr, tpr)
# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve of VGG16 Model Trained from Scratch for \n for Treatment Response Classification of \n Breast Cancer MRI Dataset', pad=10)
plt.legend(loc="lower right")

# Save the figure to a PDF file
plt.savefig('roc.pdf', bbox_inches='tight')
plt.show()

Plotting confusion matrix and calculating accuracy, specificity, and sensitivity for the VGG-16 model trained from scratch.

In [ ]:
# Read the CSV file containing predictions
df = pd.read_csv('train_from_scratch_predictions_vote.csv')

# Extract true labels and predicted probabilities
true_labels = df['Label']

# Threshold to convert probabilities to binary predictions
threshold = 0.5
binary_predictions = np.where(df['Probability'] >= threshold, classes[1], classes[0])

# Calculate confusion matrix
cm = confusion_matrix(true_labels, binary_predictions, labels=classes)

# Extract TP, TN, FP, FN from confusion matrix
tp = cm[1, 1]
tn = cm[0, 0]
fp = cm[0, 1]
fn = cm[1, 0]

# Calculate accuracy, sensitivity (True Positive Rate), and specificity (True Negative Rate)
accuracy = (tp + tn) / (tp + tn + fp + fn)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

# Display TP, TN, FP, FN
print("True Positive:", tp)
print("True Negative:", tn)
print("False Positive:", fp)
print("False Negative:", fn)

# Display accuracy, sensitivity, and specificity
print("Accuracy:", accuracy)
print("Sensitivity (True Positive Rate):", sensitivity)
print("Specificity (True Negative Rate):", specificity)

# Create a heatmap of the confusion matrix
plt.figure(figsize=(8, 6))  # Adjusted figure size for better visibility
sns.heatmap(cm, annot=True, fmt='g', cmap='magma', xticklabels=classes, yticklabels=classes)  # Using 'g' as fmt to avoid scientific notation
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title("Confusion Matrix of VGG16 Model Trained from Scratch for \n Treatment Response Classification of \n Breast Cancer MRI Dataset", pad=10)


# Save the figure to a PDF file
plt.savefig('confusion_matrix.pdf', bbox_inches='tight')

plt.show()
